In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns

%load_ext autoreload
%autoreload 2

# Retrieve the data
Download file
ftp://ftp.pride.ebi.ac.uk/pride/data/archive/2017/04/PXD005713/txt.zip
extract with p7zip

In [ ]:
# folder = Path("/media/lab/Data/Fabio/Dev/Python-InteractomeGO/data/dev-proteome-vila2017-PXD005713/txt/")
folder = Path("/media/fabio/fab/dev/Python-InteractomeGO/data/txt/")

In [ ]:
file = folder / "summary.txt"
summary = pd.read_csv(file, sep="\t", low_memory=False)
summary

In [ ]:
# list the experiments registred in the files
summary["Experiment"].unique()

In [ ]:
cohort_1 = [f"01h_{i}" for i in range(10)]
cohort_2 = [f"10h_{i}" for i in range(10)]
c_df = summary[summary["Experiment"].isin(cohort_1+cohort_2)]
p_files = c_df["Raw file"].values

fc1 = summary[summary["Experiment"].isin(cohort_1)]["Raw file"].values
fc2 = summary[summary["Experiment"].isin(cohort_2)]["Raw file"].values

### Now checking file contents to explore how the data is stored

In [ ]:
file = folder / "msms.txt"
df = pd.read_csv(file, sep="\t", low_memory=False)
df

In [ ]:
file = folder / "msScans.txt"
df = pd.read_csv(file, sep="\t", low_memory=False)
df

In [ ]:
file = folder / "msmsScans.txt"
df = pd.read_csv(file, sep="\t", low_memory=False)
df

In [ ]:
file = folder / "proteinGroups.txt"
df = pd.read_csv(file, sep="\t", low_memory=False)
df

In [ ]:
file = folder / "peptides.txt"
df = pd.read_csv(file, sep="\t")
df

In [ ]:
file = folder / "allPeptides.txt"
df = pd.read_csv(file, sep="\t")
df

My current claim is that the file that is needed to be analysed is allPeptides.txt.
Change my mind.

In [ ]:
# data massaging

# filter the experiments that we are interested in
peptides = df.query("Proteins.str.len()>1 and `Raw file`.isin(@p_files)")
# label cohorts in column 'group'
ix1 = peptides["Raw file"].isin(fc1) 
ix2 = peptides["Raw file"].isin(fc2) 
peptides.loc[ix1, "group"] = "A"
peptides.loc[ix2, "group"] = "B"
# extract replicate info from structure of the file name
peptides.loc[:, "replicate"] = peptides["Raw file"].apply(lambda f:int(f.split("_")[-1]))

peptides

In [ ]:
# check how the intensity value is calculated
# guess: average of topN entries
# conclusion: is not TopN
N = 10
ix = 3

# load array of intensities into numpy array
peptides.loc[:, "Intensities"] = peptides["Intensities"].apply(lambda i:np.fromstring(i, sep=';'))

r = peptides.iloc[ix]
intensities = sorted(r.Intensities, reverse=True)
i_guess = np.mean(intensities[:N])
print(intensities)
print(f"\n{r.Intensity:0.2f} {i_guess:0.2f} e={r.Intensity-i_guess:0.2f}")
print(f"\n{np.mean(r.Intensities):0.2f}")

In [ ]:
# transform data into log2
peptides.loc[:, "log2_int"] = np.log2(peptides["Intensity"])

ax = sns.histplot(data=peptides, x="log2_int", hue="group", bins=30)
# ax.set_xlim([0,1e13])

In [ ]:
peptides.groupby(["Proteins"]).size()

In [ ]:
r = (peptides
     .groupby(["Proteins",])
    )